In [1]:
using ExpFamilyPCA
using BenchmarkTools
using Distances

In [11]:
n = 5
indim = 3
outdim = 2
X = rand(n, indim)

ϵ = eps()
G(θ) = exp(θ)
g(θ) = exp(θ)
F(x) = x * log(x + ϵ) - x
f(x) = log(x + ϵ)
Bregman = Distances.gkl_divergence
μ = g(0);

# Benchmarking

## Default

In [12]:
@benchmark ExpFamilyPCA.fit!(
	PoissonEPCA($indim, $outdim),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 27 samples with 1 evaluation.
 Range (min … max):  180.357 ms … 302.832 ms  ┊ GC (min … max): 1.62% … 36.33%
 Time  (median):     181.893 ms               ┊ GC (median):    1.62%
 Time  (mean ± σ):   186.213 ms ±  23.323 ms  ┊ GC (mean ± σ):  3.72% ±  6.68%

  █                                                              
  █▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▁
  180 ms           Histogram: frequency by time          303 ms <

 Memory estimate: 522.61 MiB, allocs estimate: 3994150.

## EPCA 1

In [16]:
b1_Fg = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		$g,
		Val((:F, :g));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  46.410 ms … 164.967 ms  ┊ GC (min … max): 1.26% … 69.18%
 Time  (median):     48.746 ms               ┊ GC (median):    2.63%
 Time  (mean ± σ):   50.471 ms ±  11.722 ms  ┊ GC (mean ± σ):  4.72% ±  6.71%

         ▁ █▆  ▄▁▄▃   ▃     ▁▁                                  
  ▄▁▄▆▄▆▆█▆██▆▇████▆▄▇█▄▁▁▁▇██▁▇▄▁▁▇▁▁▇▄▄▁▁▁▄▁▁▁▆▁▁▁▁▁▆▁▁▁▄▄▁▄ ▄
  46.4 ms         Histogram: frequency by time         55.2 ms <

 Memory estimate: 80.50 MiB, allocs estimate: 3158424.

In [14]:
b1_Ff = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		$f,
		Val((:F, :f));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.087 s …   1.216 s  ┊ GC (min … max): 0.64% … 9.43%
 Time  (median):     1.105 s              ┊ GC (median):    0.80%
 Time  (mean ± σ):   1.123 s ± 53.233 ms  ┊ GC (mean ± σ):  2.65% ± 3.88%

  ██     █      █                                         █  
  ██▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.09 s         Histogram: frequency by time        1.22 s <

 Memory estimate: 1.50 GiB, allocs estimate: 98805865.

In [17]:
b1_F = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		Val((:F));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

MethodError: MethodError: no method matching (::ExpFamilyPCA.var"#36#37")(::Float64)
The applicable method may be too new: running in world age 31754, while current world is 31755.

Closest candidates are:
  (::ExpFamilyPCA.var"#36#37")(::Any) (method too new to be called from this world context.)
   @ ExpFamilyPCA ~/Documents/VSCodeProjects/BeliefCompression/ExpFamilyPCA/src/utils.jl:21


In [21]:
b1_FG = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$F,
		$G,
		Val((:F, :G));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

MethodError: MethodError: no method matching (::ExpFamilyPCA.var"#38#39")(::Float64)
The applicable method may be too new: running in world age 31760, while current world is 31761.

Closest candidates are:
  (::ExpFamilyPCA.var"#38#39")(::Any) (method too new to be called from this world context.)
   @ ExpFamilyPCA ~/Documents/VSCodeProjects/BeliefCompression/ExpFamilyPCA/src/utils.jl:21


## EPCA 2

In [22]:
b2_Gg = @benchmark ExpFamilyPCA.fit!(
    EPCA(
        $indim,
        $outdim,
        $G,
        $g,
        Val((:G, :g));
        μ=$μ,
        ϵ=$ϵ
    ),
	$X;
	maxiter=30,
)

InterruptException: InterruptException:

In [23]:
b2_G = @benchmark ExpFamilyPCA.fit!(
    EPCA(
        $indim,
        $outdim,
        $G,
        Val((:G));
        μ=$μ,
        ϵ=$ϵ
    ),
    $X;
    maxiter=30,
)

MethodError: MethodError: no method matching (::ExpFamilyPCA.var"#40#41")(::Float64)
The applicable method may be too new: running in world age 31765, while current world is 31766.

Closest candidates are:
  (::ExpFamilyPCA.var"#40#41")(::Any) (method too new to be called from this world context.)
   @ ExpFamilyPCA ~/Documents/VSCodeProjects/BeliefCompression/ExpFamilyPCA/src/utils.jl:21


## EPCA 3

In [24]:
b3_Bregmang = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$Bregman,
		$g,
		Val((:Bregman, :g));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

InterruptException: InterruptException:

In [25]:
b3_BregmanG = @benchmark ExpFamilyPCA.fit!(
	EPCA(
		$indim,
		$outdim,
		$Bregman,
		$G,
		Val((:Bregman, :g));
		μ=$μ,
		ϵ=$ϵ
	),
	$X;
	maxiter=30,
)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.142 s …  2.153 s  ┊ GC (min … max): 4.23% … 4.25%
 Time  (median):     2.150 s             ┊ GC (median):    4.25%
 Time  (mean ± σ):   2.148 s ± 5.558 ms  ┊ GC (mean ± σ):  4.25% ± 0.03%

  █                                       █              █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.14 s        Histogram: frequency by time        2.15 s <

 Memory estimate: 1.75 GiB, allocs estimate: 747494.

# Table

In [18]:
mean(b1_Fg).time

6.814974e9